In [1]:
import os
import shutil

# = = = = = = = = = = = = = = =

if os.path.exists('./file/4.collate'):
    shutil.rmtree('./file/4.collate')

os.makedirs('./file/4.collate')

# 1. Json

In [2]:
import pandas as pd

from tqdm import tqdm
import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/2.part/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)
    input_['Vehicle Page'] = input_['Vehicle Page'].astype(int)
    
    # = = = = = = = = = = = = = = =

    list_column, list_column_specification = input_.columns.tolist(), []

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
        df_temp, dict_dict = pd.DataFrame([input_.iloc[i]]).reset_index(drop=True), json.loads(input_.loc[i, 'Json_Specification'])
        
        for dict_ in dict_dict.values():
            for key, value in dict_.items():
                if key in list_column:
                    key = f'''{key} (Json_Specification)'''
                
                if key not in list_column_specification:
                    list_column_specification.append(key)
                    
                df_temp.loc[0, key] = value
    
        list_df.append(df_temp)
    
    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['No'],
                                ascending=[True],
                                ignore_index=True)

    output[list_column[:7] + sorted(list_column_specification) + list_column[7:]].to_excel(f'./file/4.collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 454.05it/s]



crawler_2.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 519.97it/s]


Done ~


# 2. Compatible

In [3]:
import pandas as pd

from tqdm import tqdm

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/3.vehicle/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)
    input_['Page'] = input_['Page'].astype(int)
    input_['Vehicle No'] = input_['Vehicle No'].astype(int)
    input_['Row'] = input_['Row'].astype(int)

    # = = = = = = = = = = = = = = =

    input_2 = input_.drop_duplicates(['Part Number'],
                                     keep='first',
                                     ignore_index=True)
    input_2 = input_2.sort_values(by=['No'],
                                  ascending=[True],
                                  ignore_index=True)

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
        df_temp = input_[input_['Part Number'] == input_2.loc[i, 'Part Number']].reset_index(drop=True)

        dict_vehicle = {}
        for j in range(len(df_temp)):
            make_model_engine, year = df_temp.loc[j, 'Make Model Engine'], df_temp.loc[j, 'Year']
                
            if make_model_engine not in dict_vehicle:
                dict_vehicle[make_model_engine] = [year]
            elif year not in dict_vehicle[make_model_engine]:
                dict_vehicle[make_model_engine].append(year)
    
        # = = = = = = = = = = = = = = =

        list_vehicle = []
        for make_model_engine in dict_vehicle:
            year_begin = min(dict_vehicle[make_model_engine])
            year_end = max(dict_vehicle[make_model_engine])
            list_vehicle.append(f'{make_model_engine} {year_begin}' if year_begin == year_end else f'{make_model_engine} {year_begin}-{year_end}')

        # = = = = = = = = = = = = = = =

        df_temp = pd.DataFrame([{'No': input_2.loc[i, 'No'],
                                 'Part Number': input_2.loc[i, 'Part Number'],
                                 'Vehicle': '\n'.join(sorted(list_vehicle))}])
        list_df.append(df_temp)

    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['No'],
                                ascending=[True],
                                ignore_index=True)

    output.to_excel(f'./file/4.collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1-vehicle.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 741.79it/s]



crawler_2-vehicle.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 344.77it/s]


Done ~
